## Constructing Zone 7 Load Profiles

*raw_csv* contains the following data: 
- Consumers Energy 2019 8760-hour load profile
- DTE 2018 8760-hour load profile
- DTW 2019 8760-hour load profile

Each file contains the load broken down by customer class and rate. 

My goal is to construct a 2019 8760-hour load profile for MISO Zone 7, whose shape we assume will be well-approximated by combining the 2019 Consumers Energy data with the 2019 DTE data, since these two utilities make up around 90% of the total load in zone 7. The rest is made up from many smaller utilities whose load profiles (we assume) will not dramatically affect the shape. We'll get the overall load for Zone 7 by scaling 